In [1]:
import statistics
import matplotlib.pyplot as plt
import pymysql
import config
import transformations
import ml
from sklearn.model_selection import KFold
import pandas as pd

In [2]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)

#gather all historical data to build model
RideWaits = pd.read_sql_query("call DisneyDB.RideWaitQuery", conn)

#transform data for model bulding
RideWaits = transformations.transformData(RideWaits)
RideWaits.info()

C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["MagicHourType"] = pd.Categorical(RideWaits["MagicHourType"])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 75524 entries, 0 to 78226
Data columns (total 31 columns):
RideId                 75524 non-null int8
Date                   75524 non-null datetime64[ns]
Time                   75524 non-null object
Wait                   75524 non-null int64
Name                   75524 non-null object
OpeningDate            75524 non-null datetime64[ns]
Tier                   75524 non-null category
Location               75524 non-null object
IntellectualProp       75524 non-null category
Status                 75524 non-null int8
Temperature            75524 non-null float64
CloudCover             75524 non-null float64
SimpleStatus           75524 non-null category
RainAccumulation       75524 non-null float64
ParkId                 75524 non-null category
ParkOpen               75524 non-null object
ParkClose              75524 non-null object
EMHOpen                27778 non-null object
EMHClose               27778 non-null object
ParkName      

C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days


In [3]:
originalData = RideWaits.copy()
originalData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75524 entries, 0 to 78226
Data columns (total 31 columns):
RideId                 75524 non-null int8
Date                   75524 non-null datetime64[ns]
Time                   75524 non-null object
Wait                   75524 non-null int64
Name                   75524 non-null object
OpeningDate            75524 non-null datetime64[ns]
Tier                   75524 non-null category
Location               75524 non-null object
IntellectualProp       75524 non-null category
Status                 75524 non-null int8
Temperature            75524 non-null float64
CloudCover             75524 non-null float64
SimpleStatus           75524 non-null category
RainAccumulation       75524 non-null float64
ParkId                 75524 non-null category
ParkOpen               75524 non-null object
ParkClose              75524 non-null object
EMHOpen                27778 non-null object
EMHClose               27778 non-null object
ParkName      

In [4]:
keyFeatures = ["Name","MagicHourType",
                "Tier", "IntellectualProp",
                "SimpleStatus", "ParkName",
                "DayOfWeek", "Weekend", "TimeSinceOpen", "MinutesSinceOpen",
                "CharacterExperience", "TimeSinceMidday",
                "inEMH", "EMHDay"]
newModel = ml.buildModel(RideWaits, keyFeatures, "Wait")

ValueError: labels ['Wait'] not contained in axis

In [7]:
originalData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75341 entries, 0 to 78043
Data columns (total 31 columns):
RideId                 75341 non-null int8
Date                   75341 non-null datetime64[ns]
Time                   75341 non-null object
Wait                   75341 non-null int64
Name                   75341 non-null object
OpeningDate            75341 non-null datetime64[ns]
Tier                   75341 non-null category
Location               75341 non-null object
IntellectualProp       75341 non-null category
Status                 75341 non-null int8
Temperature            75341 non-null float64
CloudCover             75341 non-null float64
SimpleStatus           75341 non-null category
RainAccumulation       75341 non-null float64
ParkId                 75341 non-null category
ParkOpen               75341 non-null object
ParkClose              75341 non-null object
EMHOpen                27659 non-null object
EMHClose               27659 non-null object
ParkName      

In [8]:
rides = originalData.Name.unique()
rides

array(['Astro Orbiter', 'Big Thunder Mountain Railroad',
       'Buzz Lightyears Space Ranger Spin', 'Dumbo the Flying Elephant',
       'Enchanted Tales with Belle', 'Haunted Mansion',
       'Its a small world', 'Jungle Cruise', 'Mad Tea Party',
       'Meet Ariel at Her Grotto',
       'Meet Cinderella and Elena at Princess Fairytale Hall',
       'Meet Cinderella and Rapunzel at Princess Fairytale Hall',
       'Meet Cinderella and a Visiting Princess at Princess Fairytale Hall',
       'Meet Daring Disney Pals as Circus Stars at Petes Silly Sideshow',
       'Meet Magnifique Disney Friends as Circus Stars at Petes Silly Sideshow',
       'Meet Mickey Mouse at Town Square Theater',
       'Meet Rapunzel and a Visiting Princess at Princess Fairytale Hall',
       'Meet Tinker Bell at Town Square Theater', 'Mickeys PhilharMagic',
       'Monsters, Inc. Laugh Floor', 'Peter Pans Flight',
       'Pirates of the Caribbean', 'Prince Charming Regal Carrousel',
       'Space Mountain', 'Sp

In [9]:
keyFeatures

['Name',
 'MagicHourType',
 'Tier',
 'IntellectualProp',
 'SimpleStatus',
 'ParkName',
 'DayOfWeek',
 'Weekend',
 'TimeSinceOpen',
 'MinutesSinceOpen',
 'CharacterExperience',
 'TimeSinceMidday',
 'inEMH',
 'EMHDay']

In [32]:
#build our new data data frame
from datetime import datetime
from datetime import timedelta
import numpy as np
today = datetime.now()
currentDate = datetime.date(today)

DayOfWeek = datetime.weekday(today)
Weekend = 1 if DayOfWeek == 5 or DayOfWeek == 6 else 0
print(today)
# print(DayOfWeek)
# print(Weekend)
newData = pd.DataFrame()
conn = pymysql.connect(config.host, user = config.username, port = config.port, passwd = config.password)

for ride in rides:
    rideData = originalData[originalData['Name'] == ride]
    rideStatic = {'Name': ride,
                  'Tier': rideData['Tier'].iloc[0],
                  'IntellectualProp': rideData['IntellectualProp'].iloc[0],
                  'ParkName': rideData['ParkName'].iloc[0],
                  'CharacterExperience': rideData['CharacterExperience'].iloc[0],
                  'DayOfWeek': DayOfWeek,
                  'Weekend': Weekend}
    rideFrame = pd.DataFrame(rideStatic, index = [0])
    getParkHours = "select * from DisneyDB.ParkHours phours join DisneyDB.Park park on phours.ParkId = park.Id where Name = '"+ rideStatic['ParkName'] + "' and Date = '" + str(currentDate)+"'"
    parkHours = pd.read_sql_query(getParkHours, conn)
    
    emhDay = 0 if parkHours.EMHOpen[0] == 'None' else 1
    rideFrame['EMHDay'] = emhDay
    parkHours['ParkOpen'] = pd.to_datetime(parkHours['ParkOpen'], format = '%I:%M %p').dt.strftime('%H:%M')
    parkHours['ParkOpen'] = pd.to_datetime(parkHours['ParkOpen'], format = '%H:%M').dt.time
    parkHours['ParkClose'] = pd.to_datetime(parkHours['ParkClose'], format = '%I:%M %p').dt.strftime('%H:%M')
    parkHours['ParkClose'] = pd.to_datetime(parkHours['ParkClose'], format = '%H:%M').dt.time
    parkHours["EMHOpen"] = pd.to_datetime(parkHours["EMHOpen"], format = '%I:%M %p', errors = 'coerce').dt.strftime('%H:%M')
    parkHours["EMHClose"] = pd.to_datetime(parkHours["EMHClose"], format = '%I:%M %p', errors = 'coerce').dt.strftime('%H:%M')
    parkHours["EMHOpen"] = pd.to_datetime(parkHours["EMHOpen"], format = '%H:%M', errors = 'coerce').dt.time
    parkHours["EMHClose"] = pd.to_datetime(parkHours["EMHClose"], format = '%H:%M', errors = 'coerce').dt.time

    parkOpen = parkHours.ParkOpen.iloc[0]
    parkClose = parkHours.ParkClose.iloc[0]
    emhOpen = parkHours.EMHOpen.iloc[0]
    emhClose = parkHours.EMHClose.iloc[0]
    
    if emhDay == 1:
        if emhClose == parkOpen:
            emhType = 'Morning'
        else:
            emhType = 'Night'
    
    pOpenToday = today.replace(hour = parkOpen.hour, minute = parkOpen.minute, second = 0, microsecond = 0)
    pCloseToday = today.replace(hour = parkClose.hour, minute= parkClose.minute, second = 0, microsecond = 0)
    if pCloseToday < pOpenToday:
        try:
            pCloseToday = pCloseToday.replace(day = pCloseToday.day + 1)
        except:
            try:
                pCloseToday = pCloseToday.replace(month = pCloseToday.month + 1, day = 1)
            except:
                pCloseToday = pCloseToday.replace(year = pCloseToday.year + 1, month = 1, day = 1)
#     print("=========================")
#     print("park open: "+ str(pOpenToday))
#     print("park close: "+ str(pCloseToday))
    if emhDay == 1:
        eOpenToday = today.replace(hour = emhOpen.hour, minute = emhOpen.minute, second = 0, microsecond = 0)
        if eOpenToday.hour < 6:
            try:
                eOpenToday = eOpenToday.replace(day = eOpenToday.day + 1)
            except:
                try:
                    eOpenToday = eOpenToday.replace(month = eOpenToday.month + 1, day = 1)
                except:
                    eOpenToday = eOpenToday.replace(year = eOpenToday.year + 1, month = 1, day = 1)
        eCloseToday = today.replace(hour = emhClose.hour, minute = emhClose.minute, second = 0, microsecond = 0)
        if (eCloseToday < pOpenToday) and (emhType == 'Night'):
            try:
                eCloseToday = eCloseToday.replace(day = eCloseToday.day + 1)
            except:
                try:
                    eCloseToday = eCloseToday.replace(month = eCloseToday.month + 1, day = 1)
                except:
                    eCloseToday = eCloseToday.replace(year = eCloseToday.year + 1, month =1, day = 1)
        print("emh open: "+ str(eOpenToday))
        print("emh close: "+ str(eCloseToday))
    
    totalRideFrame = pd.DataFrame()
    startTime = eOpenToday if emhDay == 1 and emhType == 'Morning' else pOpenToday
    validTime = True
    currentTime = startTime
    midday = today.replace(hour = 14, minute = 0, second = 0, microsecond = 0)
    while validTime:
        
        timeSinceOpen = currentTime - startTime
        timeSinceMidDay = currentTime - midday
        if emhDay == 1:
            if (currentTime >= eOpenToday) and (currentTime <= eCloseToday):
                inEMH = 1
            else:
                inEMH = 0
        else:
            inEMH = 0
        
        minutesSinceOpen = int(round(timeSinceOpen.total_seconds()/60))
        timeSinceMidDayHours =  int(round(abs(timeSinceMidDay.total_seconds()/3600)))
        timeSinceOpenHours = int(round(timeSinceOpen.total_seconds()/3600))
        
        currentRow = rideFrame.copy()
        currentRow['TimeSinceOpen'] = timeSinceOpenHours
        currentRow['MinutesSinceOpen'] = minutesSinceOpen
        currentRow['TimeSinceMidday'] = timeSinceMidDayHours
        currentRow['inEMH'] = inEMH
        
        totalRideFrame = pd.concat([totalRideFrame,currentRow])
        
        newTime = currentTime + timedelta(minutes=15)
        if emhDay == 1:
            if emhType == 'Morning':
                if (newTime >= eOpenToday) and (newTime <= pCloseToday):
                    validTime = True
                else:
                    validTime = False
            else:
                if (newTime <= pOpenToday) and (newTime <= eCloseToday):
                    validTime = True
                else:
                    validTime = False
        else:
            if (newTime >= pOpenToday) and (newTime <= pCloseToday):
                validTime = True
            else:
                validTime = False
        currentTime = newTime
    newData = pd.concat([newData, totalRideFrame])
#     print([startTime, endTime,emhDay, inEMH])
conn.close()
    #print(parkHours)
    

2018-06-01 14:53:24.065667
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:00
emh open: 2018-06-01 07:00:00
emh close: 2018-06-01 08:00:

In [11]:
newData

,CharacterExperience,DayOfWeek,IntellectualProp,Name,ParkName,Tier,Weekend,EMHDay,TimeSinceOpen,MinutesSinceOpen,TimeSinceMidday,inEMH
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,0,0,7,1
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,0,15,7,1
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,0,30,6,1
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,1,45,6,1
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,1,60,6,1
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,1,75,6,0
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,2,90,6,0
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,2,105,5,0
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,2,120,5,0
0,0,4,None,Astro Orbiter,Magic Kingdom,minor_attraction,0,1,2,135,5,0


In [12]:
newData[newData["Name"] == 'The Boneyard']

,CharacterExperience,DayOfWeek,IntellectualProp,Name,ParkName,Tier,Weekend,EMHDay,TimeSinceOpen,MinutesSinceOpen,TimeSinceMidday,inEMH
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,0,0,6,1
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,0,15,6,1
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,0,30,6,1
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,1,45,5,1
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,1,60,5,1
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,1,75,5,0
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,2,90,4,0
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,2,105,4,0
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,2,120,4,0
0,0,4,None,The Boneyard,Animal Kingdom,diversion,0,1,2,135,4,0


In [13]:
newData["SimpleStatus"] = "Clear"

In [14]:
newModel

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=60,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=4, min_weight_fraction_leaf=0.0,
           n_estimators=1800, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [31]:
import sys
!{sys.executable} -m pip install statsmodels
from statsmodels.tools import categorical


,CharacterExperience,DayOfWeek,EMHDay,IntellectualProp,MinutesSinceOpen,Name,ParkName,SimpleStatus,Tier,TimeSinceMidday,...,headliner,inEMH,major_attraction,minor_attraction,super_headliner,diversion,headliner,major_attraction,minor_attraction,super_headliner
0,0.0,4.0,1.0,5.0,0.0,0.0,Magic Kingdom,Clear,minor_attraction,7.0,...,NaN,1.0,NaN,NaN,NaN,0,0,0,1,0
0,0.0,4.0,1.0,5.0,15.0,0.0,Magic Kingdom,Clear,minor_attraction,7.0,...,NaN,1.0,NaN,NaN,NaN,0,0,0,1,0
0,0.0,4.0,1.0,5.0,30.0,0.0,Magic Kingdom,Clear,minor_attraction,6.0,...,NaN,1.0,NaN,NaN,NaN,0,0,0,1,0
0,0.0,4.0,1.0,5.0,45.0,0.0,Magic Kingdom,Clear,minor_attraction,6.0,...,NaN,1.0,NaN,NaN,NaN,0,0,0,1,0
0,0.0,4.0,1.0,5.0,60.0,0.0,Magic Kingdom,Clear,minor_attraction,6.0,...,NaN,1.0,NaN,NaN,NaN,0,0,0,1,0
0,0.0,4.0,1.0,5.0,75.0,0.0,Magic Kingdom,Clear,minor_attraction,6.0,...,NaN,0.0,NaN,NaN,NaN,0,0,0,1,0
0,0.0,4.0,1.0,5.0,90.0,0.0,Magic Kingdom,Clear,minor_attraction,6.0,...,NaN,0.0,NaN,NaN,NaN,0,0,0,1,0
0,0.0,4.0,1.0,5.0,105.0,0.0,Magic Kingdom,Clear,minor_attraction,5.0,...,NaN,0.0,NaN,NaN,NaN,0,0,0,1,0
0,0.0,4.0,1.0,5.0,120.0,0.0,Magic Kingdom,Clear,minor_attraction,5.0,...,NaN,0.0,NaN,NaN,NaN,0,0,0,1,0
0,0.0,4.0,1.0,5.0,135.0,0.0,Magic Kingdom,Clear,minor_attraction,5.0,...,NaN,0.0,NaN,NaN,NaN,0,0,0,1,0


In [20]:
RideWaits.info()
RideWaits.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75341 entries, 0 to 78043
Data columns (total 31 columns):
RideId                 75341 non-null int8
Date                   75341 non-null datetime64[ns]
Time                   75341 non-null object
Wait                   75341 non-null int64
Name                   75341 non-null int8
OpeningDate            75341 non-null datetime64[ns]
Tier                   75341 non-null int8
Location               75341 non-null object
IntellectualProp       75341 non-null int8
Status                 75341 non-null int8
Temperature            75341 non-null float64
CloudCover             75341 non-null float64
SimpleStatus           75341 non-null int8
RainAccumulation       75341 non-null float64
ParkId                 75341 non-null int8
ParkOpen               75341 non-null object
ParkClose              75341 non-null object
EMHOpen                27659 non-null object
EMHClose               27659 non-null object
ParkName               75341 non